# Template 

- Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)

In [1]:
%load_ext watermark

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd

import sys
sys.path.append('../src/random_forest_mc')
from model import *


In [38]:
# from tqdm import tqdm

from glob import glob

# import matplotlib.pyplot as plt
# %matplotlib inline
# from matplotlib import rcParams
# from cycler import cycler

# rcParams['figure.figsize'] = 12, 8 # 18, 5
# rcParams['axes.spines.top'] = False
# rcParams['axes.spines.right'] = False
# rcParams['axes.grid'] = True
# rcParams['axes.prop_cycle'] = cycler(color=['#365977'])
# rcParams['lines.linewidth'] = 2.5

# import seaborn as sns
# sns.set_theme()

# pd.set_option("max_columns", None)
# pd.set_option("max_rows", None)
# pd.set_option('display.max_colwidth', None)

from IPython.display import Markdown, display
def md(arg):
    display(Markdown(arg))

# from pandas_profiling import ProfileReport
# #report = ProfileReport(#DataFrame here#, minimal=True)
# #report.to

# import pyarrow.parquet as pq
# #df = pq.ParquetDataset(path_to_folder_with_parquets, filesystem=None).read_pandas().to_pandas()

import json
def open_file_json(path,mode='r',var=None):
    if mode == 'w':
        with open(path,'w') as f:
            json.dump(var, f)
    if mode == 'r':
        with open(path,'r') as f:
            return json.load(f)

# import functools
# import operator
# def flat(a):
#     return functools.reduce(operator.iconcat, a, [])

# import json
# from glob import glob
# from typing import NewType


# DictsPathType = NewType("DictsPath", str)


# def open_file_json(path):
#     with open(path, "r") as f:
#         return json.load(f)

# class LoadDicts:
#     def __init__(self, dict_path: DictsPathType = "./data"):
#         Dicts_glob = glob(f"{dict_path}/*.json")
#         self.List = []
#         self.Dict = {}
#         for path_json in Dicts_glob:
#             name = path_json.split("/")[-1].replace(".json", "")
#             self.List.append(name)
#             self.Dict[name] = open_file_json(path_json)
#             setattr(self, name, self.Dict[name])


In [4]:
# Run this cell before close.
%watermark -d --iversion -b -r -g -m -v
!cat /proc/cpuinfo |grep 'model name'|head -n 1 |sed -e 's/model\ name/CPU/'
!free -h |cut -d'i' -f1  |grep -v total

Python implementation: CPython
Python version       : 3.9.6
IPython version      : 7.26.0

Compiler    : GCC 10.2.1 20210110
OS          : Linux
Release     : 5.11.0-7620-generic
Machine     : x86_64
Processor   : 
CPU cores   : 4
Architecture: 64bit

Git hash: 64c542d41d5900d7c5ccdfd2f1ec1bab094328a4

Git repo: https://github.com/ysraell/random-forest-mc.git

Git branch: main

pandas: 1.3.2
numpy : 1.21.2
sys   : 3.9.6 (default, Aug 17 2021, 15:37:33) 
[GCC 10.2.1 20210110]

CPU	: Intel(R) Core(TM) i7-7500U CPU @ 2.70GHz
Mem:            15G
Swap:          4.0G


In [5]:
dataset_path_list = glob('/work/tmp/datasets/*.csv')
dataset_path_list

['/work/tmp/datasets/titanic.csv',
 '/work/tmp/datasets/creditcard_trans_float.csv',
 '/work/tmp/datasets/creditcard.csv',
 '/work/tmp/datasets/creditcard_trans_int.csv',
 '/work/tmp/datasets/iris.csv']

In [40]:
dataset_dict = {
    'titanic' : {
        'ds_cols' : ['Pclass', 'Sex', 'Age', 'SibSp', 'Fare', 'Embarked'],
        'target_col' : 'Survived',
        'csv_path' : '/work/tmp/datasets/titanic.csv'
    },
    'iris': {
        'ds_cols' : ['sepal.length', 'sepal.width', 'petal.length', 'petal.width'],
        'target_col' : 'variety',
        'csv_path' : '/work/tmp/datasets/iris.csv'
    },
    'creditcard': {
        'ds_cols' : ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount'],
        'target_col' : 'Class',
        'csv_path' : '/work/tmp/datasets/creditcard.csv'
    },
    'creditcard_trans_int': {
        'ds_cols' : ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount'],
        'target_col' : 'Class',
        'csv_path' : '/work/tmp/datasets/creditcard_trans_int.csv'
    },
    'creditcard_trans_float': {
        'ds_cols' : ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount'],
        'target_col' : 'Class',
        'csv_path' : '/work/tmp/datasets/creditcard_trans_float.csv'
    }
}

open_file_json('../tests/datasets_metadata.json',mode='w',var=dataset_dict)

In [7]:
ds_name = 'titanic'
md(f'# {ds_name}')
params = dataset_dict[ds_name]

dataset = pd.read_csv(params['csv_path'])[params['ds_cols']+[params['target_col']]].dropna().reset_index(drop=True)
dataset['Age'] = dataset['Age'].astype(np.uint8)
dataset['SibSp'] = dataset['SibSp'].astype(np.uint8)
dataset['Pclass'] = dataset['Pclass'].astype(str)
dataset['Fare'] = dataset['Fare'].astype(np.uint32)
cls = RandomForestMC(
    n_trees=8,
    target_col = params['target_col'],
    max_discard_trees = 4
)
cls.process_dataset(dataset)
cls.fit()
y_test = dataset[params['target_col']].to_list()
y_pred = cls.testForest(dataset)
accuracy_hard = "{:.2f}\%".format(100*sum([v == p for v, p in zip(y_test, y_pred)]) / len(y_pred))
y_pred = cls.testForest(dataset, soft_voting=True)
accuracy_soft = "{:.2f}\%".format(100*sum([v == p for v, p in zip(y_test, y_pred)]) / len(y_pred))
md('## Accuracy:') 
md(f'## &nbsp;&nbsp;&nbsp; {accuracy_hard} (hard-voting)')
md(f'## &nbsp;&nbsp;&nbsp; {accuracy_soft} (soft-voting)')

# titanic

Planting the forest: 100%|██████████| 8/8 [00:06<00:00,  1.28it/s]


## Accuracy:

## &nbsp;&nbsp;&nbsp; 78.23\% (hard-voting)

## &nbsp;&nbsp;&nbsp; 78.09\% (soft-voting)

In [12]:
ds_name = 'iris'
md(f'# {ds_name}')
params = dataset_dict[ds_name]

dataset = pd.read_csv(params['csv_path'])[params['ds_cols']+[params['target_col']]].dropna().reset_index(drop=True)
dataset.rename(columns={col: col.replace('.','_') for col in dataset.columns}, inplace=True)
params['ds_cols'] = [col.replace('.','_') for col in params['ds_cols']]
cls = RandomForestMC(
    n_trees=8,
    target_col = params['target_col'],
    max_discard_trees = 4
)
cls.process_dataset(dataset)
cls.fit()
y_test = dataset[params['target_col']].to_list()
y_pred = cls.testForest(dataset)
accuracy_hard = "{:.2f}\%".format(100*sum([v == p for v, p in zip(y_test, y_pred)]) / len(y_pred))
y_pred = cls.testForest(dataset, soft_voting=True)
accuracy_soft = "{:.2f}\%".format(100*sum([v == p for v, p in zip(y_test, y_pred)]) / len(y_pred))
md('## Accuracy:') 
md(f'## &nbsp;&nbsp;&nbsp; {accuracy_hard} (hard-voting)')
md(f'## &nbsp;&nbsp;&nbsp; {accuracy_soft} (soft-voting)')

# iris

Planting the forest: 100%|██████████| 8/8 [00:11<00:00,  1.45s/it]


## Accuracy:

## &nbsp;&nbsp;&nbsp; 38.00\% (hard-voting)

## &nbsp;&nbsp;&nbsp; 38.00\% (soft-voting)

In [35]:
for ds_name in ['creditcard', 'creditcard_trans_int', 'creditcard_trans_float']:
    md(f'# {ds_name}')
    params = dataset_dict[ds_name]

    dataset = pd.read_csv(params['csv_path'])[params['ds_cols']+[params['target_col']]].dropna().reset_index(drop=True)
    cls = RandomForestMC(
        n_trees=8,
        target_col = params['target_col'],
        max_discard_trees = 4
    )
    cls.process_dataset(dataset)
    cls.fit()
    dataset = dataset.sample(n=1000)
    y_test = dataset[params['target_col']].to_list()
    y_pred = cls.testForest(dataset)
    accuracy_hard = "{:.2f}\%".format(100*sum([v == p for v, p in zip(y_test, y_pred)]) / len(y_pred))
    y_pred = cls.testForest(dataset, soft_voting=True)
    accuracy_soft = "{:.2f}\%".format(100*sum([v == p for v, p in zip(y_test, y_pred)]) / len(y_pred))
    md('## Accuracy:') 
    md(f'## &nbsp;&nbsp;&nbsp; {accuracy_hard} (hard-voting)')
    md(f'## &nbsp;&nbsp;&nbsp; {accuracy_soft} (soft-voting)')

# creditcard

Planting the forest: 100%|██████████| 8/8 [00:10<00:00,  1.26s/it]


## Accuracy:

## &nbsp;&nbsp;&nbsp; 0.30\% (hard-voting)

## &nbsp;&nbsp;&nbsp; 0.30\% (soft-voting)

...

# creditcard_trans_float

Planting the forest: 100%|██████████| 8/8 [00:10<00:00,  1.33s/it]


## Accuracy:

## &nbsp;&nbsp;&nbsp; 0.10\% (hard-voting)

## &nbsp;&nbsp;&nbsp; 0.10\% (soft-voting)